In [30]:
import pandas as pd
import numpy as np
import math


In [31]:
# read in data
bids = pd.read_csv('bid_data_wo_me.csv')
bids.columns = [f"v_{i}" for i in range(10,101,10)]
my_bids = [9,11,11,21,21,31,31,41,41,51]
# my_bids = [9,11,21,31,41,51,61,71,81,91]

In [3]:
bids.head()

,v_10,v_20,v_30,v_40,v_50,v_60,v_70,v_80,v_90,v_100
0,5.0,15.0,20.0,30.0,35.0,45.0,60.0,60.0,70.0,80.0
1,5.0,10.0,15.0,20.0,25.0,30.0,35.0,40.0,45.0,50.0
2,10.0,20.0,30.0,40.0,50.0,60.0,70.0,80.0,90.0,100.0
3,5.0,10.0,15.0,20.0,25.0,30.0,35.0,40.0,45.0,50.0
4,5.0,10.0,20.0,30.0,40.0,50.0,60.0,70.0,80.0,90.0


# Part 1
## 1 Calculate your winning probability and expected utility with your bids submitted in Ex 1.2 for each of your values.

Event $W_v$ = prob the bid for my value v wins the auction
$$
Pr(W_v) = \frac{|mybid > allbids|}{|allbids|} \forall v \in [10...100]
$$
coinflip on mybid == otherbid

In [51]:
# my_bids = [9,11,11,21,21,31,31,41,41,51]
my_bids = [10,11,21,31,41,51,61,71,81,91]

In [52]:
# calculating percentage wins for my bids (if equal then i win)
tot_bids = bids.count().sum()
my_bids_win_rate = [0]*10
for i, my_bid in enumerate(my_bids):
    strictlywin = (bids < my_bid).sum().sum()
    # should i math.floor the below?
    tiewins = (bids == my_bid).sum().sum() * 0.5
    
    my_bids_win_rate[i] = round((strictlywin + tiewins)/ tot_bids, 4)
my_bids_win_rate

[0.111, 0.1768, 0.3049, 0.4293, 0.5561, 0.6744, 0.8012, 0.8671, 0.9232, 0.9756]

note, with coinflip for bid ties, overall slightly lower win rate (makes sense)

Expected utility for bid for each value is prob of that bid winning and the profit for such value and bid
$$
E[U_v] = Pr(w_v)*(v - b_v)
$$

In [53]:
# calc expected utility
my_bids_ex_utility = [0]*10
for i, my_bid in enumerate(my_bids):
    value = (i+1)*10
    my_bids_ex_utility[i] = round(my_bids_win_rate[i] * (value - my_bid), 4)
my_bids_ex_utility

[0.0, 1.5912, 2.7441, 3.8637, 5.0049, 6.0696, 7.2108, 7.8039, 8.3088, 8.7804]

### MC Sim

In [45]:
# my_bids = [9,11,11,21,21,31,31,41,41,51]
my_bids = [10,11,21,31,41,51,61,71,81,91]

In [46]:
def MCSimProbWin(my_bid, bids, niters=100):
    bidsmatrix = bids.to_numpy()
    onedimbids = bidsmatrix.flatten()
    win_count = 0
    
    for _ in range(niters):
        competing_bid = np.random.choice(onedimbids)
        if competing_bid < my_bid:
            win_count += 1
        elif competing_bid == my_bid:
            # coin flip
            if np.random.randint(0,2) == 0:
                win_count += 1
#     print(win_count)
    return round(win_count/niters, 4)

In [47]:
# by mc sim
[MCSimProbWin(b_v, bids, 10000) for b_v in my_bids]

[0.1105,
 0.1835,
 0.2982,
 0.4314,
 0.5457,
 0.6868,
 0.7989,
 0.8655,
 0.9236,
 0.9757]

In [48]:
my_bids

[10, 11, 21, 31, 41, 51, 61, 71, 81, 91]

In [49]:
def MCSimEU(my_bid, value, bids, niters=1000):
    bidsmatrix = bids.to_numpy()
    onedimbids = bidsmatrix.flatten()
    totprofit = 0
    
    for _ in range(niters):
        competing_bid = np.random.choice(onedimbids)
        if competing_bid < my_bid:
            totprofit += value-my_bid
        elif competing_bid == my_bid:
            # coin flip
            if np.random.randint(0,2) == 0:
                totprofit += value-my_bid
    return round(totprofit/niters, 4)

In [78]:
my_bids = [9,11,11,21,21,31,31,41,41,51]
# my_bids = [10,11,21,31,41,51,61,71,81,91]

In [79]:
# by mc sim
[MCSimEU(b_v, (i+1)*10, bids, 10000) for i, b_v in enumerate(my_bids)]

[0.0505,
 1.5651,
 3.3022,
 5.8919,
 8.9552,
 12.3395,
 16.419,
 21.7308,
 27.5282,
 32.9427]

^ quite different from exact calculations. Also not consistent between values, suggests that for some values peoples' bids are more "rational" than others?

## 2 Calculate the optimal bid which maximizes your expected utility given the distribution of opponent bids.

$$
OPTbid_v = \underset{b}{\operatorname{argmax}} Pr(W_v) * (v-b_v)
$$
<!-- \argmax pr(w_v)*(v-b_v)
$$ -->


In [10]:
# gives best whole integer bid
def euargmax(v, bids, granularity=1):
    tot_bids = bids.count().sum()
    argmaxb = None
    curmaxeu = -1
    bidrange = np.arange(1,v, granularity)
#     print(bidrange)
    for b in bidrange:
        strictwin = (bids < b).sum().sum()
        tiewin = (bids == b).sum().sum() * 0.5
        prob_win = (strictwin + tiewin) / tot_bids
        gain = v-b
        eu = prob_win*gain
        if eu > curmaxeu:
            curmaxeu = prob_win*gain
            argmaxb = b
#             print(round(b,3))
    return argmaxb,3, curmaxeu

In [11]:
euargmax(10, bids, 0.0001)

(5.000099999999559, 3, 0.2073129268292866)

In [12]:
# optimal bids
[euargmax(v, bids, 0.1) for v in range(10,101,10)]

[(5.0000000000000036, 3, 0.20731707317073156),
 (11.000000000000009, 3, 1.6902439024390228),
 (11.000000000000009, 3, 3.568292682926828),
 (21.000000000000018, 3, 5.978048780487799),
 (21.000000000000018, 3, 9.124390243902434),
 (31.00000000000003, 3, 12.731707317073159),
 (31.00000000000003, 3, 17.121951219512184),
 (41.000000000000036, 3, 22.068292682926806),
 (41.000000000000036, 3, 27.72682926829266),
 (51.00000000000004, 3, 33.58292682926827)]

Interestingly, after accounting for tie situations and having a finer granularity, optimal bid did not change.

## 3 Compare the utility you obtained to the optimal utility you could have obtained.  Can you conclude anything about a good strategy in this auction?

My utility is the same as the optimal utility except for when value = 10. In this case I think it's because people were not playing optimally so they created space for profit even for when value = 10. In the optimal game, a player with value=10 vs another player with v>10 would lose and yield eu=0 and facing an opponent with v=10 would do no better. Either a bid of 10 to yield 0 profit or a bid of 9 in the case of a tie, win 1/2 of $1 profit. It also could be interpreted as for items of low value to bidders, despite knowing that you may have the lowest valuation and most likely would not win against someone else, a low ball offer just might beat someone else's higher lowball offer. When I was working on this, I realized that it was a matter of looking for the greatest utility given the probabilties of winning so doing some rough calculations i was able to get the same optimal strategic bids as the one found emprically. 

# Part 2
Consider devising a bidding algorithm from data.  Suppose you had a small sample of bid data; how would you use this data to devise a good bidding strategy?  How does a good algorithm change with the amount of data you have?  Suppose you have 1 sample?  or 10 samples?  Or what if you have 100 samples?  Evaluate your bidding algorithm using Monte Carlo simulation.


If we had some data, we can do something similar to what we did in Part 1. With more amounts of data, the better our algorithm can be trained to pinpoint a more precise value at which a bid can win. 
With 1 sample, it's really a shot at the dark but in reality this may be the case. For instance I'm taking a course in the kellogg real estate department and strip malls in distress for example may be auctioned off. There's the concept of a stalking horse bid which at least sets an anchor on the valuation of the asset. So for our algorithm, just having 1 piece of data can help since instead of randomly choosing a number, it can just take that SH_bid and bid + 1. 

With 10 samples we may get a clearer picture of the mean, the stdevation, and range of the bids. For 100 samples it should be very clear to have the algorithm bid correctly since in the case of the class example there was 41 samples per value (disregarding one's own's bid) and it achieved an accuracy in EU calculation of up to X decimal places.



In [13]:
v30np = bids.to_numpy().flatten()

In [14]:
(np.random.choice(v30np, 100) < 20).sum()

25

In [15]:
def bid_algo(v, data, samples=10, granularity=1):
    # from data, output a bid_v for v.
    
    # what does a sample look like?
    # a cross section or 1 instance
    # assuming samples are for the given v (or else if 1 sample and other v != my_v, then doesn't help me at all)
#     sample_vector = data[f'v_{v}'].to_numpy()
#     sampled_view = np.random.choice(sample_vector, samples)
    
    sample_vector = data.to_numpy()
    sampled_view = np.random.choice(sample_vector.flatten(), samples)
    
#     print(sampled_view)
    
    # learn the optimal bid from data using euargmax
    argmaxb = None
    curmaxeu = -1
    bidrange = np.arange(1,v, granularity)

    for b in bidrange:
        strictwin = (sampled_view < b).sum()
        tiewin = (sampled_view == b).sum() * 0.5
        prob_win = (strictwin + tiewin) / samples
        gain = v-b
        eu = prob_win*gain
        if eu > curmaxeu:
            curmaxeu = prob_win*gain
            argmaxb = b
#             print(round(b,3))
    return round(argmaxb,3), round(curmaxeu,3)
    

In [16]:
bid_algo(80, bids, samples=100)

(41, 23.79)

In [17]:
for value in range(10,101,10):
    for samples in [1, 10, 100, 1000]:
        optb, eu = bid_algo(value, bids, samples=samples)
        print(f'best bid, exp util when v={value} with {samples} samples:             ', optb, eu)
    print('')

best bid, exp util when v=10 with 1 samples:              1 0.0
best bid, exp util when v=10 with 10 samples:              1 0.0
best bid, exp util when v=10 with 100 samples:              1 0.18
best bid, exp util when v=10 with 1000 samples:              6 0.136

best bid, exp util when v=20 with 1 samples:              1 0.0
best bid, exp util when v=20 with 10 samples:              11 2.7
best bid, exp util when v=20 with 100 samples:              11 1.665
best bid, exp util when v=20 with 1000 samples:              11 1.755

best bid, exp util when v=30 with 1 samples:              1 0.0
best bid, exp util when v=30 with 10 samples:              21 4.5
best bid, exp util when v=30 with 100 samples:              11 3.42
best bid, exp util when v=30 with 1000 samples:              12 3.564

best bid, exp util when v=40 with 1 samples:              1 0.0
best bid, exp util when v=40 with 10 samples:              22 1.8
best bid, exp util when v=40 with 100 samples:              22 5.